In [1]:
import random
import numpy as np
import torch

import mmcv
import mmrotate
import mmdet
from mmcv.ops import get_compiling_cuda_version, get_compiler_version
# Check mmcv installation
print(mmcv.__version__) #noqa E1101
# Check MMRotate installation
print(mmrotate.__version__)
print(mmrotate.__file__)
# Check MMDetection installation
print(mmdet.__version__)

print(get_compiling_cuda_version())
print(get_compiler_version())

2.0.0
1.0.0rc1
/mnt/hdd/eric/.tmp_ipy/15.Lab_Detection/06.Gits/01.SAMRS_MTP/MTP/MMRotate_1_x/mmrotate/mmrotate/__init__.py
3.1.0
11.1
GCC 7.3


In [2]:
import mmengine
print(mmengine.__version__)

0.10.3


In [3]:
from mmengine.config import Config
from mmengine.runner import Runner

from mmdet.utils import register_all_modules as register_all_modules_mmdet
from mmdet.apis import inference_detector, init_detector

from mmrotate.registry import VISUALIZERS
from mmrotate.utils import register_all_modules

In [4]:
from mmrotate.registry import MODELS
from mmrotate.testing import get_detector_cfg
#from torchinfo import summary

In [5]:
from mmrotate.registry import DATASETS
from mmrotate.datasets.dota import DOTADataset

@DATASETS.register_module()
class ShipDataset(DOTADataset):
    """SAR ship dataset for detection."""
    
    METAINFO = {
            'classes':('ship',),
                # palette is a list of color tuples, which is used for visualization.
            'palette': [(165, 42, 42),]
        }

In [7]:
cfg_path = "/mnt/hdd/eric/.tmp_ipy/15.Lab_Detection/06.Gits/01.SAMRS_MTP/MTP/MMRotate_1_x/fine_tune/oriented_rcnn_rvsa_l_1024_mae_mtp_dota10_adjust.py"
cfg = Config.fromfile(cfg_path)

cfg_name = cfg_path.split("/")[-1].replace(".py", "_v1")

# Modify dataset type and path
cfg.data_root = '/mnt/hdd/eric/.tmp_ipy/00.Data/ShipRS_dataset/ShipRSImageNet_V1/DOTA_Format_MMrotate_1x/'
cfg.dataset_type = 'ShipDataset'

# modify num classes of the model in box head
cfg['model']['roi_head']['bbox_head']['num_classes']=1
# We can still use the pre-trained Mask RCNN model though we do not need to
# use the mask branch
cfg.load_from = "/mnt/hdd/eric/.tmp_ipy/00.Weights/MTP/diorr-rvsa-l-mae-mtp-epoch_12.pth"
cfg.checkpoint = cfg.load_from
cfg['model']['backbone']['pretrained'] = None

# Set up working dir to save files and logs.
cfg.work_dir = "/mnt/hdd/eric/.tmp_ipy/15.Lab_Detection/06.Gits/01.SAMRS_MTP/MTP/MMRotate_1_x/work_dir"

cfg.train_cfg.val_interval = 1
# Change the evaluation metric since we use customized dataset.
cfg.val_evaluator.metric = 'mAP'
# We can set the evaluation interval to reduce the evaluation times
cfg.val_evaluator.interval = 1
# We can set the checkpoint saving interval to reduce the storage cost
cfg.default_hooks.checkpoint.interval = 1
# cfg.max_epochs=36
cfg.train_cfg.max_epochs=12

# Set seed thus the results are more reproducible
cfg.seed = 0
random.seed(cfg.seed)
np.random.seed(cfg.seed)
torch.cuda.manual_seed(cfg.seed)
torch.cuda.manual_seed_all(cfg.seed)

cfg.gpu_ids = range(1)
cfg.device='cuda:0'


# modify pipeline mainly for resize scale (512,512)
cfg.train_pipeline = [
    dict(type='mmdet.LoadImageFromFile', backend_args=None),
    dict(type='mmdet.LoadAnnotations', with_bbox=True, box_type='qbox'),
    dict(type='ConvertBoxType', box_type_mapping=dict(gt_bboxes='rbox')),
    dict(type='mmdet.Resize', scale=(800, 800), keep_ratio=True),
    dict(
        type='mmdet.RandomFlip',
        prob=0.75,
        direction=['horizontal', 'vertical', 'diagonal']),
    dict(
        type='mmdet.Pad', size=(800, 800),
        pad_val=dict(img=(114, 114, 114))),
    
    dict(type='mmdet.PackDetInputs')
]
cfg.val_pipeline = [
    dict(type='mmdet.LoadImageFromFile', backend_args=None),
    dict(type='mmdet.Resize', scale=(800, 800), keep_ratio=True),
    dict(type='mmdet.LoadAnnotations', with_bbox=True, box_type='qbox'),
    dict(type='ConvertBoxType', box_type_mapping=dict(gt_bboxes='rbox')),
    dict(
        type='mmdet.Pad', size=(800, 800),
        pad_val=dict(img=(114, 114, 114))),
    dict(
        type='mmdet.PackDetInputs',
        meta_keys=('img_id', 'img_path', 'ori_shape', 'img_shape',
                   'scale_factor'))
]
cfg.test_pipeline = [
    dict(type='mmdet.LoadImageFromFile', backend_args=None),
    dict(type='mmdet.Resize', scale=(800, 800), keep_ratio=True),
    dict(
        type='mmdet.Pad', size=(800, 800),
        pad_val=dict(img=(114, 114, 114))),
    dict(
        type='mmdet.PackDetInputs',
        meta_keys=('img_id', 'img_path', 'ori_shape', 'img_shape',
                   'scale_factor'))
]


cfg.val_dataloader.dataset.type = cfg.dataset_type
cfg.val_dataloader.dataset.ann_file = 'val'
cfg.val_dataloader.dataset.data_prefix=dict(img_path='images_png', img='images_png')
# cfg.val_dataloader.dataset.img_prefix = 'images'
cfg.val_dataloader.dataset.data_root = '/mnt/hdd/eric/.tmp_ipy/00.Data/ShipRS_dataset/ShipRSImageNet_V1/DOTA_Format_MMrotate_1x/'

cfg.train_dataloader.dataset.type = cfg.dataset_type
cfg.train_dataloader.dataset.ann_file = 'train'
cfg.train_dataloader.dataset.data_prefix=dict(img_path='images_png', img='images_png')
# cfg.train_dataloader.dataset.img_prefix = 'images'
cfg.train_dataloader.dataset.data_root = '/mnt/hdd/eric/.tmp_ipy/00.Data/ShipRS_dataset/ShipRSImageNet_V1/DOTA_Format_MMrotate_1x/'

cfg.test_dataloader.dataset.type = cfg.dataset_type
cfg.test_dataloader.dataset.data_prefix=dict(img_path='images_png', img='images_png')
cfg.test_dataloader.dataset.ann_file = 'val'
# cfg.test_dataloader.dataset.img_prefix = 'images'
cfg.test_dataloader.dataset.data_root = '/mnt/hdd/eric/.tmp_ipy/00.Data/ShipRS_dataset/ShipRSImageNet_V1/DOTA_Format_MMrotate_1x/'

cfg.train_dataloader.dataset.pipeline=cfg.train_pipeline
cfg.val_dataloader.dataset.pipeline=cfg.val_pipeline
cfg.test_dataloader.dataset.pipeline=cfg.test_pipeline

cfg.val_evaluator = dict(type='DOTAMetric', metric='mAP')
cfg.test_evaluator = cfg.val_evaluator
# We can also use tensorboard to log the training process
# cfg.vis_backends=[dict(type='TensorboardVisBackend')]
# cfg.visualizer = dict(type='mmrotate.RotLocalVisualizer',vis_backends=vis_backends)

# We can initialize the logger for training and have a look
# at the final config used for training


#----- wrapper
cfg.pop("optim_wrapper")

optimizer = dict(type='AdamW', lr=0.001, momentum=0.9, weight_decay=0.0001)
optim_wrapper = dict(type='OptimWrapper', optimizer=optimizer)
cfg['optim_wrapper'] = optim_wrapper

cfg.pop('param_scheduler')
cfg['optim_wrapper'].pop('constructor')
cfg['optim_wrapper'].pop('paramwise_cfg')

#-----
cfg.dump(f'./work_dir/{cfg_name}.py')
print(f'Config:\n{cfg.pretty_text}')

Config:
angle_version = 'le90'
backend_args = None
checkpoint = '/mnt/hdd/eric/.tmp_ipy/00.Weights/MTP/diorr-rvsa-l-mae-mtp-epoch_12.pth'
data_root = '/mnt/hdd/eric/.tmp_ipy/00.Data/ShipRS_dataset/ShipRSImageNet_V1/DOTA_Format_MMrotate_1x/'
dataset_type = 'ShipDataset'
default_hooks = dict(
    checkpoint=dict(interval=1, type='CheckpointHook'),
    logger=dict(interval=50, type='LoggerHook'),
    param_scheduler=dict(type='ParamSchedulerHook'),
    sampler_seed=dict(type='DistSamplerSeedHook'),
    timer=dict(type='IterTimerHook'),
    visualization=dict(type='mmdet.DetVisualizationHook'))
default_scope = 'mmrotate'
device = 'cuda:0'
env_cfg = dict(
    cudnn_benchmark=False,
    dist_cfg=dict(backend='nccl'),
    mp_cfg=dict(mp_start_method='fork', opencv_num_threads=0))
gpu_ids = range(0, 1)
load_from = '/mnt/hdd/eric/.tmp_ipy/00.Weights/MTP/diorr-rvsa-l-mae-mtp-epoch_12.pth'
log_level = 'INFO'
log_processor = dict(by_epoch=True, type='LogProcessor', window_size=50)
model = dict(
  

# Train a detector

In [12]:
# register all modules in mmdet into the registries
# do not init the default scope here because it will be init in the runner
register_all_modules_mmdet(init_default_scope=False)
register_all_modules(init_default_scope=False)

# register all modules in mmrotate into the registries
register_all_modules()

# build the model from a config file and a checkpoint file
model = init_detector(cfg, cfg.checkpoint, palette="dota", device=cfg.device)

runner = Runner.from_cfg(cfg)
runner.train()

/mnt/hdd/eric/.conda/envs/8.tmp/lib/python3.8/site-packages/torch/functional.py:445: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at  ../aten/src/ATen/native/TensorShape.cpp:2157.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]


The relative_pos_embedding is used
The relative_pos_embedding is used
The relative_pos_embedding is used
The relative_pos_embedding is used
The relative_pos_embedding is used
The relative_pos_embedding is used
The relative_pos_embedding is used
The relative_pos_embedding is used
The relative_pos_embedding is used
The relative_pos_embedding is used
The relative_pos_embedding is used
The relative_pos_embedding is used
The relative_pos_embedding is used
The relative_pos_embedding is used
The relative_pos_embedding is used
The relative_pos_embedding is used
The relative_pos_embedding is used
The relative_pos_embedding is used
The relative_pos_embedding is used
The relative_pos_embedding is used
Loads checkpoint by local backend from path: /mnt/hdd/eric/.tmp_ipy/00.Weights/MTP/diorr-rvsa-l-mae-mtp-epoch_12.pth
The model and loaded state dict do not match exactly

size mismatch for roi_head.bbox_head.fc_cls.weight: copying a param with shape torch.Size([21, 1024]) from checkpoint, the shape 

/mnt/hdd/eric/.tmp_ipy/15.Lab_Detection/06.Gits/01.SAMRS_MTP/MTP/MMRotate_1_x/mmrotate/mmrotate/structures/bbox/rotated_boxes.py:192: UserWarning: The `clip` function does nothing in `RotatedBoxes`.
  warnings.warn('The `clip` function does nothing in `RotatedBoxes`.')
/mnt/hdd/eric/.tmp_ipy/15.Lab_Detection/06.Gits/01.SAMRS_MTP/MTP/MMRotate_1_x/mmrotate/mmrotate/structures/bbox/rotated_boxes.py:192: UserWarning: The `clip` function does nothing in `RotatedBoxes`.
  warnings.warn('The `clip` function does nothing in `RotatedBoxes`.')
/mnt/hdd/eric/.tmp_ipy/15.Lab_Detection/06.Gits/01.SAMRS_MTP/MTP/MMRotate_1_x/mmrotate/mmrotate/structures/bbox/rotated_boxes.py:192: UserWarning: The `clip` function does nothing in `RotatedBoxes`.
  warnings.warn('The `clip` function does nothing in `RotatedBoxes`.')
/mnt/hdd/eric/.tmp_ipy/15.Lab_Detection/06.Gits/01.SAMRS_MTP/MTP/MMRotate_1_x/mmrotate/mmrotate/structures/bbox/rotated_boxes.py:192: UserWarning: The `clip` function does nothing in `Rotate

04/24 17:29:01 - mmengine - INFO - Epoch(train)  [1][  50/1374]  lr: 1.0000e-04  eta: 5:26:07  time: 1.1904  data_time: 0.0114  memory: 10955  loss: 0.4044  loss_rpn_cls: 0.0608  loss_rpn_bbox: 0.0598  loss_cls: 0.2069  acc: 95.8984  loss_bbox: 0.0769
04/24 17:30:00 - mmengine - INFO - Epoch(train)  [1][ 100/1374]  lr: 1.0000e-04  eta: 5:23:21  time: 1.1774  data_time: 0.0036  memory: 10955  loss: 0.2891  loss_rpn_cls: 0.0428  loss_rpn_bbox: 0.0569  loss_cls: 0.1182  acc: 98.2422  loss_bbox: 0.0712
04/24 17:31:00 - mmengine - INFO - Epoch(train)  [1][ 150/1374]  lr: 1.0000e-04  eta: 5:23:19  time: 1.1944  data_time: 0.0035  memory: 10955  loss: 0.2841  loss_rpn_cls: 0.0492  loss_rpn_bbox: 0.0730  loss_cls: 0.1001  acc: 94.0430  loss_bbox: 0.0618
04/24 17:31:59 - mmengine - INFO - Epoch(train)  [1][ 200/1374]  lr: 1.0000e-04  eta: 5:22:20  time: 1.1876  data_time: 0.0035  memory: 10955  loss: 0.2943  loss_rpn_cls: 0.0272  loss_rpn_bbox: 0.0663  loss_cls: 0.1238  acc: 91.6992  loss_bbox:

/mnt/hdd/eric/.conda/envs/8.tmp/lib/python3.8/site-packages/torch/utils/checkpoint.py:25: UserWarning: None of the inputs have requires_grad=True. Gradients will be None
  warnings.warn("None of the inputs have requires_grad=True. Gradients will be None")


...

/mnt/hdd/eric/.conda/envs/8.tmp/lib/python3.8/site-packages/torch/utils/checkpoint.py:25: UserWarning: None of the inputs have requires_grad=True. Gradients will be None
  warnings.warn("None of the inputs have requires_grad=True. Gradients will be None")


04/24 22:56:04 - mmengine - INFO - Epoch(val) [11][ 50/276]    eta: 0:01:35  time: 0.4217  data_time: 0.0048  memory: 10955  
04/24 22:56:25 - mmengine - INFO - Epoch(val) [11][100/276]    eta: 0:01:14  time: 0.4258  data_time: 0.0031  memory: 8178  
04/24 22:56:46 - mmengine - INFO - Epoch(val) [11][150/276]    eta: 0:00:53  time: 0.4192  data_time: 0.0031  memory: 8178  
04/24 22:57:07 - mmengine - INFO - Epoch(val) [11][200/276]    eta: 0:00:31  time: 0.4172  data_time: 0.0031  memory: 8178  
04/24 22:57:28 - mmengine - INFO - Epoch(val) [11][250/276]    eta: 0:00:10  time: 0.4237  data_time: 0.0031  memory: 8178  
04/24 22:57:39 - mmengine - INFO - 
---------------iou_thr: 0.5---------------
04/24 22:57:42 - mmengine - INFO - 
+-------+------+------+--------+-------+
| class | gts  | dets | recall | ap    |
+-------+------+------+--------+-------+
| ship  | 2501 | 5655 | 0.895  | 0.793 |
+-------+------+------+--------+-------+
| mAP   |      |      |        | 0.793 |
+-------+----

/mnt/hdd/eric/.conda/envs/8.tmp/lib/python3.8/site-packages/torch/utils/checkpoint.py:25: UserWarning: None of the inputs have requires_grad=True. Gradients will be None
  warnings.warn("None of the inputs have requires_grad=True. Gradients will be None")


04/24 23:26:35 - mmengine - INFO - Epoch(val) [12][ 50/276]    eta: 0:01:36  time: 0.4291  data_time: 0.0048  memory: 10955  
04/24 23:26:57 - mmengine - INFO - Epoch(val) [12][100/276]    eta: 0:01:15  time: 0.4234  data_time: 0.0033  memory: 8178  
04/24 23:27:18 - mmengine - INFO - Epoch(val) [12][150/276]    eta: 0:00:53  time: 0.4258  data_time: 0.0032  memory: 8178  
04/24 23:27:40 - mmengine - INFO - Epoch(val) [12][200/276]    eta: 0:00:32  time: 0.4381  data_time: 0.0032  memory: 8178  
04/24 23:28:01 - mmengine - INFO - Epoch(val) [12][250/276]    eta: 0:00:11  time: 0.4275  data_time: 0.0032  memory: 8178  
04/24 23:28:12 - mmengine - INFO - 
---------------iou_thr: 0.5---------------
04/24 23:28:15 - mmengine - INFO - 
+-------+------+------+--------+-------+
| class | gts  | dets | recall | ap    |
+-------+------+------+--------+-------+
| ship  | 2501 | 4776 | 0.894  | 0.781 |
+-------+------+------+--------+-------+
| mAP   |      |      |        | 0.781 |
+-------+----

FasterRCNN(
  (data_preprocessor): DetDataPreprocessor()
  (backbone): RVSA_MTP_branches(
    (patch_embed): PatchEmbed(
      (proj): Conv2d(3, 1024, kernel_size=(16, 16), stride=(16, 16))
    )
    (pos_drop): Dropout(p=0.0, inplace=False)
    (blocks): ModuleList(
      (0): Block(
        (norm1): LayerNorm((1024,), eps=1e-06, elementwise_affine=True)
        (attn): RotatedVariedSizeWindowAttention(
          (sampling_offsets): Sequential(
            (0): AvgPool2d(kernel_size=7, stride=7, padding=0)
            (1): LeakyReLU(negative_slope=0.01)
            (2): Conv2d(1024, 32, kernel_size=(1, 1), stride=(1, 1))
          )
          (sampling_scales): Sequential(
            (0): AvgPool2d(kernel_size=7, stride=7, padding=0)
            (1): LeakyReLU(negative_slope=0.01)
            (2): Conv2d(1024, 32, kernel_size=(1, 1), stride=(1, 1))
          )
          (sampling_angles): Sequential(
            (0): AvgPool2d(kernel_size=7, stride=7, padding=0)
            (1): Leak